<font size=6>
    <b>Read_conllu_files.ipynb:</b> Read and parse information from diverse .conllu files, and use integrations with libraries to apply data efficently
 </font>

## Introduction

This notebook demonstrates how diverse .conllu files can be imported, converted and worked with using the open source library [Text Extensions for Pandas](https://github.com/CODAIT/text-extensions-for-pandas). This library uses [Pandas](https://pandas.pydata.org/) DataFrames as a primary data storage format, and to work with several different NLP libraries, such as [SpaCy](https://spacy.io), [Huggingface Transformers](https://huggingface.co/transformers/). 

Here we show how these features can be used in conjunction to import, select data, display sentence structure information, and then finally retokenize and train a classifier model on the dataset. 



In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json
import feather
import sklearn.pipeline
import sklearn.linear_model
import transformers

# And of course we need the text_extensions_for_pandas library itself.
try:
    import text_extensions_for_pandas as tp
except ModuleNotFoundError as e:
    # If we're running from within the project source tree and the parent Python
    # environment doesn't have the text_extensions_for_pandas package, use the
    # version in the local source tree.
    if not os.getcwd().endswith("notebooks"):
        raise e
    if ".." not in sys.path:
        sys.path.insert(0, "..")
    import text_extensions_for_pandas as tp
    

### Loading files
There are several sub-flavors of .conllu files, including those used in the EWT, Ontonotes, Universal Dependencies, and  CoNLL 2009 corpuses. Text Extensions is designed to take advantage of the common features of .conllu files, while allowing for varied types to be accepted. 

In importing this file type, we
1. Translate the raw words into Token Dtypes
1. Preserve the dependencies between tokens as represented in the `head` and `deprel` columns
1. Capture conllu metadata written into the file, if it exists 
1. Allow for conll 09 and Ontonotes style predicate - predicate argument representations
1. Capture each token's sentence 
1. Allow the user to choose how sub-tokens are handled


First, though we must load the datasets we will be using for this demo notebook 

In the following cell, we use the facilities of Text Extensions for Pandas to download a copy of the [Universal Dependencies EWT data set](https://github.com/UniversalDependencies/UD_English-EWT) and the [Trial section of the CoNLL 2009 dataset](https://ufal.mff.cuni.cz/conll2009-st/trial-data.html). **Make sure that you adhere to the terms under which they are liscensed when using them** 

Then we read them in and display them in the document. Notice how different the information stored in each dataset is. One thing to note is in this specific example, we drop a few columns from each dataset for brevity; remove the `.drop()` methods to show more lines. 

In [55]:
# init file locations, and download data if necessary. 
BASE_DIR = 'CoNLL_u_test_inputs/'
FEATHER_FILE = "conllu_database.feather"

ewt_base_url = "https://github.com/UniversalDependencies/UD_English-EWT/blob/master/en_ewt-ud-"
ewt_dev_url = ewt_base_url + 'dev.conllu'
conll_09_test_data_url =  'https://ufal.mff.cuni.cz/conll2009-st/trial/CoNLL2009-ST-English-trial.zip'

# allows us to re-start from saved points
corpus_df = None 

In [2]:
# download the files if they have not already been downloaded 
conll_09_path = tp.io.conll.maybe_download_dataset_data(BASE_DIR, conll_09_test_data_url)
conllu_ewt_path = tp.io.conll.maybe_download_dataset_data(BASE_DIR, ewt_dev_url)

# if you already have access to the full conll 2009 dataset, name the file accordingly and uncomment this line 
# conll_09_path = BASE_DIR + 'CoNLL2009-ST-evaluation-English.conllu'

In [40]:
# import two very different documents, both in the conllu file format. 

# by default we look for EWT style column names, 
# so we have to define a new set for this specific conll09 format
conll_09_cols = ["LEMMA","PLEMMA",'POS','PPOS','FEAT','PFEAT','head','phead','DEPREL','PDEPREL','FILLPRED','PRED']

conll_09_docs = tp.io.conll.conll_u_to_dataframes(conll_09_path,column_names=conll_09_cols)
#now just filter,and display the document 
conll_09_doc = conll_09_docs[0].drop(columns=["PLEMMA",'PPOS','PFEAT','phead','PDEPREL','FILLPRED','sentence','line_num'])
print("Conll 09 format .conllu document:")
display(conll_09_doc.head())


#simultaneously, we can import an ewt style document, and display it with the same function
conll_u_docs = tp.io.conll.conll_u_to_dataframes(conllu_ewt_path)
#display 
DOC_NUM = 0
doc_df = conll_u_docs[DOC_NUM]
# here we drop the sentence argument for brevity.
print("EWT format .conllu document:")
doc_df.head(10).drop(columns = ["sentence"])

Conll 09 format .conllu document:


,span,LEMMA,POS,FEAT,head,DEPREL,PRED,predicate,pred0arg,pred1arg,pred2arg,pred3arg,pred4arg,pred5arg,pred6arg,pred7arg,pred8arg,pred9arg,pred10arg
0,"[0, 3): 'The'",the,DT,None,1.0,NMOD,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"[4, 11): 'economy'",economy,NN,None,3.0,NMOD,None,A1,None,None,None,None,None,None,None,None,None,None,None
2,"[11, 13): ''s'",'s,POS,None,1.0,SUFFIX,None,None,None,None,None,None,None,None,None,None,None,None,None
3,"[14, 25): 'temperature'",temperature,NN,None,4.0,SBJ,temperature.01,A2,A1,None,None,None,None,None,None,None,None,None,None
4,"[26, 30): 'will'",will,MD,None,NaN,ROOT,None,None,AM-MOD,None,None,None,None,None,None,None,None,None,None


EWT format .conllu document:


,span,lemma,upostag,xpostag,features,head,deprel,deps,misc,sentence_id,paragraph_id,doc_id,line_num
0,"[0, 4): 'From'",from,ADP,IN,None,2.0,case,3:case,None,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,4
1,"[5, 8): 'the'",the,DET,DT,Definite=Def|PronType=Art,2.0,det,3:det,None,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,5
2,"[9, 11): 'AP'",AP,PROPN,NNP,Number=Sing,3.0,obl,4:obl:from,None,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,6
3,"[12, 17): 'comes'",come,VERB,VBZ,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...,NaN,root,0:root,None,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,7
4,"[18, 22): 'this'",this,DET,DT,Number=Sing|PronType=Dem,5.0,det,6:det,None,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,8
5,"[23, 28): 'story'",story,NOUN,NN,Number=Sing,3.0,nsubj,4:nsubj,None,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,9
6,"[28, 29): ':'",:,PUNCT,:,None,3.0,punct,4:punct,None,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,10
7,"[30, 39): 'President'",President,PROPN,NNP,Number=Sing,11.0,nsubj,5:nsubj,None,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,15
8,"[40, 44): 'Bush'",Bush,PROPN,NNP,Number=Sing,7.0,flat,1:flat,None,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,16
9,"[45, 47): 'on'",on,ADP,IN,None,10.0,case,4:case,None,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,weblog-blogspot.com_nominations_20041117172713...,17


## Combining documents and saving as a .feather file. 

Something that is often useful is to store multiple documents from a corpus in one single dataset. Here we do that, then make an adjustment to keep the `'head'` column of our database pointing at the correct elements.


Next we quickly write then reread our document as a `.feather` file in its. manipulated state. Because this is serialized, writing and reading is significantly faster than writing to a raw `.conllu` format.  

In [48]:
# because we are concatenating our dataframes, we need to modify the "head" feilds to still point at their desired targets 
df_starts_at =0
temp = conll_u_docs.copy()
for df in temp: 
    df["head"] += df_starts_at
    df_starts_at +=df.shape[0]

# Now concatenate all our documents into one big dataframe
complete_df = temp[0]
complete_df = complete_df.append(temp[1:], ignore_index=True)

#show the last few rows of the dataframe, select just a few columns for compactness
print(f"size is {complete_df.shape[0]}")
complete_df.tail()[["span","lemma","upostag","head","deprel"]]

size is 25151


,span,lemma,upostag,head,deprel
25146,"[251, 254): 'and'",and,CCONJ,100432.0,cc
25147,"[255, 256): 'a'",a,DET,100432.0,det
25148,"[257, 261): 'very'",very,ADV,100431.0,advmod
25149,"[262, 275): 'knowledgeable'",knowledgeable,ADJ,100432.0,amod
25150,"[276, 281): 'staff'",staff,NOUN,100427.0,conj


In [49]:
# one advantage of using pandas dataframes is that we can write and read them signifcantly faster than we could the raw conllu files 
# here we use pyarrow with feather to save and reload our dataframe. 

# Currently writing multi document files is not supported, so we will have to use a workaround, 
# by converting sentences from TokenSpanArrays to SpanArrays
complete_df["sentence"] = tp.SpanArray(complete_df["span"].array.target_text, complete_df["sentence"].array.begin, complete_df["sentence"].array.end)

#finally write to file using feather 
path = BASE_DIR +FEATHER_FILE
# increase the chunksize slightly, to allow writing in a single block
feather.write_dataframe(complete_df, path,chunksize= 65536*8)
print(f"File written to {path}")

File written to CoNLL_u_test_inputs/conllu_database.feather


In [50]:
# now we can read this df and continue operating on it as before
re_read_df = feather.read_dataframe(path)
print(f"size is {re_read_df.shape[0]}")
# show the same subset of the dataframe as above 
re_read_df.tail()[["span","lemma","upostag","head","deprel"]]

size is 25151


,span,lemma,upostag,head,deprel
25146,"[251, 254): 'and'",and,CCONJ,100432.0,cc
25147,"[255, 256): 'a'",a,DET,100432.0,det
25148,"[257, 261): 'very'",very,ADV,100431.0,advmod
25149,"[262, 275): 'knowledgeable'",knowledgeable,ADJ,100432.0,amod
25150,"[276, 281): 'staff'",staff,NOUN,100427.0,conj


## Show sentence parse trees using pandas data manipulation, and SpaCy integrations
Because of the integrations built into Text extensions, we can use powerful data visualization tools here we're leveraging spaCy's dependency tree visualization tools, to show the parse tree as specified in the raw conllu file. 

First, we use Pandas groupby to to quickly select the n'th sentence in the dataset, and store it as its own dataframe and display selected columns 

Then we use Spacy to render the parse tree of that specific sentence, as found in the raw data. 

In [51]:
Sentence_num = 110

# use pandas to quickly select the 'n'th sentence in the dataset 
nth_sentence = list(re_read_df.groupby("sentence_id",sort=False))[Sentence_num][1]
display(nth_sentence[["span","lemma","upostag","xpostag","head","deprel","sentence"]])

# now use spacy integration to rendeer the parse tree
tp.io.spacy.render_parse_tree(first_sentence,tag_col="upostag",label_col="deprel",head_col="head")

,span,lemma,upostag,xpostag,head,deprel,sentence
2510,"[979, 982): 'And'",and,CCONJ,CC,9471.0,cc,"[979, 1014): 'And what do we get for this effo..."
2511,"[983, 987): 'what'",what,PRON,WP,9471.0,obj,"[979, 1014): 'And what do we get for this effo..."
2512,"[988, 990): 'do'",do,AUX,VBP,9471.0,aux,"[979, 1014): 'And what do we get for this effo..."
2513,"[991, 993): 'we'",we,PRON,PRP,9471.0,nsubj,"[979, 1014): 'And what do we get for this effo..."
2514,"[994, 997): 'get'",get,VERB,VB,NaN,root,"[979, 1014): 'And what do we get for this effo..."
2515,"[998, 1001): 'for'",for,ADP,IN,9474.0,case,"[979, 1014): 'And what do we get for this effo..."
2516,"[1002, 1006): 'this'",this,DET,DT,9474.0,det,"[979, 1014): 'And what do we get for this effo..."
2517,"[1007, 1013): 'effort'",effort,NOUN,NN,9471.0,obl,"[979, 1014): 'And what do we get for this effo..."
2518,"[1013, 1014): '?'",?,PUNCT,.,9471.0,punct,"[979, 1014): 'And what do we get for this effo..."


# Train a classifier model

Now use more text extensions integrations, with *transformers* to quickly and easily train a part of speech classifier model using bert embeddings on our data. We loosely follow the same process as is used in the [Model_Training_with_BERT](./Model_Training_with_BERT.ipynb) demo, notebook so check there for a more indepth explanation of each step.

Broadly, what we do is: 
1. Import all the folds of the dataset we're using (Universal dependencies EWT) 
1. Create a Pandas Categorical datatype on over which to classify
1. Retokenize that dataset using Huggingface Transformers to Bert-compatible tokens
1. Correlate the new tokens with their original counterpart's parts of speech
1. Create the Bert embeddings for each sub-token
1. Convert the parts of speech tags to our categoical datatype
1. Initialize and train a sklearn model on the Bert embeddings -> Part of Speech
1. Use that model to perform inference on our dataset



In [8]:
# We're going to need the whole ewt dataset for this: download them, and parse them in 
fold_paths = {"test":  tp.io.conll.maybe_download_dataset_data(BASE_DIR, ewt_base_url + "test.conllu"),
              "dev":   tp.io.conll.maybe_download_dataset_data(BASE_DIR, ewt_base_url + "dev.conllu"),
              "train": tp.io.conll.maybe_download_dataset_data(BASE_DIR, ewt_base_url + "train.conllu")}
fold_docs = {}
for fold,fold_path in fold_paths.items(): 
    fold_docs[fold] = tp.io.conll.conll_u_to_dataframes(fold_path)
    print(f"converted fold: '{fold}' to list of {len(fold_docs[fold])} dataframes")
    #     uncomment to display segments of the extracted folds 
    #     display(fold_docs[fold][0].head()[['span','lemma','upostag','features','sentence']])


converted fold: 'test' to list of 316 dataframes
converted fold: 'dev' to list of 318 dataframes
converted fold: 'train' to list of 540 dataframes


### Initialize elements for preprocessing steps
Instantiate pretrained tokenizer and BERT models from transformers library, and create a pandas categorical datatype for parts of speech

In [52]:
bert_model_name = "dslim/bert-base-NER"
tokenizer = transformers.BertTokenizerFast.from_pretrained(bert_model_name)
bert = transformers.BertModel.from_pretrained(bert_model_name)

# also we will want to create a pandas categorical dtype for what we want to predict- part of speech. 
# use the combined df, because it has all the elements 
upostags_list = list(re_read_df["upostag"].unique())
# upostag_dtype,upostag_list,upostag_dict = tp.io.conll.make_iob_tag_categories(upostags)
upostag_dtype = pd.CategoricalDtype(categories = upostags_list)
upostag_dict = {upostags_list[i]:i for i in range(len(upostags_list)) }

## Preprocess the document

Because steps 3-6 can only be done on a document-by-document basis, we create a method to do them in a batch, then run them  on the whole corpus. Note this process is computationally intensive so it may take a few minutes to run.

In [11]:
# make a method to take care of preprocessing steps: 3-6
def preprocess_document(document, tokenizer,bert):
    # create BERT compatible tokens using our tokenizer
    temp = tp.io.bert.make_bert_tokens(document.loc[0,'span'].target_text, tokenizer)
    # re-correlate our original spans with their bert-compatible equivalents
    spans = tp.TokenSpanArray.align_to_tokens(temp["span"],document["span"])
    temp["postag"] = tp.io.conll.spans_to_iob(spans,document["upostag"]).drop(columns = ["ent_iob"])
    # now translate from text tags to postag 
    temp['postag'].fillna('X',inplace=True) # in our Labels, 'X' is a standin for "N/A" so convert N/A's to 'X'
    temp['postag'].astype(upostag_dtype,copy=False)
    temp["postag_id"] = temp['postag'].apply(lambda t: upostag_dict[str(t)])
    return tp.io.bert.add_embeddings(temp, bert)

# now preprocess the whole corpus: 
bert_docs_by_fold = {}
for fold in fold_docs.keys():
    docs = fold_docs[fold]
    print(f"processing fold {fold}")
    bert_docs_by_fold[fold] = tp.jupyter.run_with_progress_bar(len(docs),lambda i: preprocess_document(docs[i],tokenizer,bert))

## Checkpoint: save preprocessed data 

Because the last step was time intensive, combine all the documents together, then save them as a feather file, so that we can restart from here if need be. 

In [57]:
# combine folds and save to a feather file, so we don't necessarily need to redo the preprocessing. 
corpus_df = tp.io.conll.combine_folds(bert_docs_by_fold)
corpus_df["text"] = corpus_df["span"].apply(lambda s: s.covered_text)
corpus_df.drop(columns=["span"]).to_feather("outputs/conll_u_corpus.feather")
corpus_df.head()

,fold,doc_num,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,postag,postag_id,embedding,text
0,test,0,0,"[0, 0): ''",101,0,1,True,X,14,"[ -0.37686592, -0.14841378, 0.73980016, ...",
1,test,0,1,"[0, 4): 'What'",1327,0,1,False,PRON,11,"[ -0.23266968, -0.40546328, 0.6171929, ...",What
2,test,0,2,"[5, 7): 'if'",1191,0,1,False,SCONJ,13,"[ -0.8156859, -0.04782569, 0.081484295, ...",if
3,test,0,3,"[8, 14): 'Google'",7986,0,1,False,PROPN,2,"[ 0.78967804, -0.8511879, -0.48812625, ...",Google
4,test,0,4,"[15, 17): 'Mo'",12556,0,1,False,VERB,3,"[ -0.25935018, 0.5710723, -0.09106647, ...",Mo


In [14]:
# re-read feather document if need be: 
if corpus_df is None or corpus_df.size == 0:
    corpus_df = pd.read_feather("outputs/conll_u_corpus.feather")
    corpus_df

In [58]:
# now get ready to train our model: 
train_df = corpus_df[corpus_df["fold"] == "train"]
train_df.head()

,fold,doc_num,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,postag,postag_id,embedding,text
64729,train,0,0,"[0, 0): ''",101,0,1,True,X,14,"[ -0.41927838, -0.22575253, 0.6648760...",
64730,train,0,1,"[0, 2): 'Al'",2586,0,1,False,PROPN,2,"[ -0.36961424, -1.0804733, -0.283367...",Al
64731,train,0,2,"[2, 3): '-'",118,0,1,False,PUNCT,5,"[ -0.9178737, -0.94624436, -0.808995...",-
64732,train,0,3,"[4, 5): 'Z'",163,0,1,False,PROPN,2,"[ -0.90530086, -0.97086835, -1.440879...",Z
64733,train,0,4,"[5, 9): 'aman'",19853,0,1,False,PROPN,2,"[ -1.1586123, -1.149766, -1.194975...",aman


In [16]:
# now actually train a model, using sklearn 
MULTI_CLASS= "multinomial"

# How many iterations to run the BGFS optimizer when fitting logistic
# regression models. 100 ==> Fast; 10000 ==> Full convergence
LBGFS_ITERATIONS = 1000

base_pipeline = sklearn.pipeline.Pipeline([
    # Standard scaler. This only makes a difference for certain classes
    # of embeddings.
    #("scaler", sklearn.preprocessing.StandardScaler()),
    ("mlogreg", sklearn.linear_model.LogisticRegression(
        multi_class=MULTI_CLASS,
        verbose=10,
        max_iter=LBGFS_ITERATIONS
    ))
])

X_train = train_df["embedding"].values
Y_train = train_df["postag_id"]
base_model = base_pipeline.fit(X_train, Y_train)
base_model

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/zacharyeichenberger/anaconda3/envs/pd/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 15.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 15.6min finished


Pipeline(steps=[('mlogreg',
                 LogisticRegression(max_iter=1000, multi_class='multinomial',
                                    verbose=10))])

In [17]:
# pickle model so I don't need to re-fit it every time
import pickle 
pickle_model_file = "conllu_pos_classifier.pickle"
with open(BASE_DIR+ pickle_model_file, 'wb') as file: 
    pickle.dump(base_model,file)


### Use the model to run inference on the test set of the data 

In [60]:
def infer_on_df(df: pd.DataFrame, id_to_class_dict, predictor):
    result_df = df.copy()
    raw_outputs = tp.TensorArray(predictor.predict_proba(result_df["embedding"]))
    result_df["p_id"] = np.argmax(raw_outputs, axis=1)
    result_df["p_postag"]= result_df["p_id"].apply(lambda p_id: id_to_class_dict[p_id])
#     iobs, types = tp.io.conll.decode_class_labels(result_df ["p_postag"].values)
#     result_df["p_iob"] = iobs
#     result_df["p_type"] = types
    result_df["raw_output"] = raw_outputs
    return result_df

dev_results = infer_on_df(corpus_df[corpus_df["fold"] == "dev"],upostags_list,base_model)
dev_results.head(10)

,fold,doc_num,token_id,span,input_id,token_type_id,attention_mask,special_tokens_mask,postag,postag_id,embedding,text,p_id,p_postag,raw_output
32680,dev,0,0,"[0, 0): ''",101,0,1,True,X,14,"[ -0.3136824, -0.12475453, 0.657083...",,14,X,"[ 3.759111492243688e-08, 3.5428922757842324e-0..."
32681,dev,0,1,"[0, 4): 'From'",1622,0,1,False,ADP,0,"[ -0.16610569, 0.0027155988, 0.836163...",From,0,ADP,"[ 0.9887720094502591, 4.9811565390014984e-0..."
32682,dev,0,2,"[5, 8): 'the'",1103,0,1,False,DET,1,"[ -0.5103949, -0.43374223, 0.522710...",the,1,DET,"[ 5.466092360822045e-07, 0.999994439172927..."
32683,dev,0,3,"[9, 11): 'AP'",10997,0,1,False,PROPN,2,"[ -0.6617647, -0.4930782, -0.01439116...",AP,2,PROPN,"[1.8021154562937207e-07, 1.756215071415866e-1..."
32684,dev,0,4,"[12, 17): 'comes'",2502,0,1,False,VERB,3,"[ -0.12817244, -0.1552504, 0.5966705...",comes,3,VERB,"[ 0.04499116598493901, 0.0001437495683970017..."
32685,dev,0,5,"[18, 22): 'this'",1142,0,1,False,DET,1,"[ -0.2173722, -0.19165185, 0.821224...",this,8,ADJ,"[ 0.0035047614732790612, 0.290947742370616..."
32686,dev,0,6,"[23, 28): 'story'",1642,0,1,False,NOUN,4,"[ -0.18569958, -0.2532726, 0.7620976...",story,4,NOUN,"[ 0.005917029813905384, 1.1698890474344557e-0..."
32687,dev,0,7,"[28, 29): ':'",131,0,1,False,PUNCT,5,"[ -0.23127511, -0.5130299, 0.9697360...",:,5,PUNCT,"[ 0.0002293081082192774, 6.227019009117806e-0..."
32688,dev,0,8,"[30, 39): 'President'",1697,0,1,False,PROPN,2,"[ -0.4024123, -0.15248768, 1.034773...",President,2,PROPN,"[ 7.213782089605227e-05, 2.0248820667712914e-0..."
32689,dev,0,9,"[40, 44): 'Bush'",6096,0,1,False,PROPN,2,"[ -0.5243888, 0.1511954, 0.6104687...",Bush,2,PROPN,"[ 5.792456141255749e-21, 1.9232206223508995e-2..."
